In [2]:
import datetime as dt
import json as js
import urllib.request
import sqlite3 as sql
url='https://api.openweathermap.org/data/2.5/onecall?lat=64.286670&lon=27.622125&units=metric&appid=e25afeccd6e22a2e996bed2809e43452'

resp=urllib.request.urlopen(url)
jsfile_Tornio=js.load(resp)

def unix_to_UTC_time(time_survey):
    return dt.datetime.utcfromtimestamp(time_survey).strftime('%Y-%m-%d %H:%M:%S')

date_weather_info=[]
for each_day in jsfile_Tornio['daily']:
    date_predicted=unix_to_UTC_time(each_day['dt'])
    temp_max=each_day['temp']['max']
    temp_min=each_day['temp']['min']
    temp_avg=each_day['temp']['day']
    rain_info=each_day.get('rain',0) # if it was no rain on the surveyed day, then return value 0

    date_weather_info.append({
        'date_predicted':date_predicted,
        'temp_max':temp_max,
        'temp_min':temp_min,
        'temp_avg':temp_avg,
        'rain_info (mm)': rain_info
    })


try:
    con=sql.connect('WeatherDataTest.db')
    ori_table=con.execute('select * from WeatherPrediction').fetchall()
    #if there is no table named 'WeatherPrediction' in 'WeatherDataTest.db', then jump to except statement
    if len(ori_table) != 0:
        k=ori_table[-1][0]+1
    else:
        k=1
except:
    k=1

data_weather=[]
today=dt.datetime.now().strftime('%Y-%m-%d')
for i in date_weather_info:
    data_tuple=(k,today,i['date_predicted'],i['temp_min'],i['temp_max'],i['temp_avg'],i['rain_info (mm)'])
    data_weather.append(data_tuple)
    k+=1
con.commit()
con.close()
data_weather

[(57, '2020-04-16', '2020-04-16 10:00:00', -0.37, 1.23, 0, 0),
 (58, '2020-04-16', '2020-04-17 10:00:00', -1.52, 1.22, 0.62, 0),
 (59, '2020-04-16', '2020-04-18 10:00:00', -0.53, 2.39, 1.77, 0),
 (60, '2020-04-16', '2020-04-19 10:00:00', -1.87, 3.27, 2.25, 0),
 (61, '2020-04-16', '2020-04-20 10:00:00', 1.24, 6.03, 4.36, 0),
 (62, '2020-04-16', '2020-04-21 10:00:00', 1.33, 8.73, 7.13, 0),
 (63, '2020-04-16', '2020-04-22 10:00:00', 2.76, 7.71, 7.41, 0),
 (64, '2020-04-16', '2020-04-23 10:00:00', 1.6, 7.29, 6.73, 0)]

In [3]:
import sqlite3 as sql
Wea_pred=sql.connect('WeatherDataTest.db')
try:   
    Wea_pred.execute('''
    CREATE TABLE "WeatherPrediction" (
        "prediction_id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        "predictionDate"	DATE NOT NULL,
        "date"	DATE NOT NULL,
        "minTemp"	INTEGER NOT NULL DEFAULT 0,
        "maxTemp"	INTEGER NOT NULL DEFAULT 0,
        "avgTemp"	INTEGER NOT NULL DEFAULT 0,
        "rain"	INTEGER NOT NULL DEFAULT 0
    );
    ''')
except:
    pass
    
try:
    Wea_pred.executemany('''insert into WeatherPrediction (prediction_id, predictionDate, date, minTemp, maxTemp, avgTemp, rain)        values (?,?, ?, ?, ?, ?, ?)''',data_weather)
except:
    pass
Wea_pred.commit()
Wea_pred.close()


In [4]:
%%capture
%load_ext sql
%sql sqlite:///WeatherDataTest.db

In [5]:
%%sql
select * from WeatherPrediction


* sqlite:///WeatherDataTest.db
Done.


[(1, '2020-04-09', '2020-04-09 10:00:00', -4.05, 2.62, 2.62, 0),
 (2, '2020-04-09', '2020-04-10 10:00:00', -5.62, -0.78, -2.32, 0),
 (3, '2020-04-09', '2020-04-11 10:00:00', -4.41, 0.64, -0.43, 0),
 (4, '2020-04-09', '2020-04-12 10:00:00', -4.06, 0.29, -0.98, 0),
 (5, '2020-04-09', '2020-04-13 10:00:00', -4.73, -0.92, -0.92, 0),
 (6, '2020-04-09', '2020-04-14 10:00:00', -7.14, -1.27, -3.53, 0),
 (7, '2020-04-09', '2020-04-15 10:00:00', -1.86, 2.73, 2.21, 0.92),
 (8, '2020-04-09', '2020-04-16 10:00:00', -3.35, 4.45, 2.73, 0.3),
 (9, '2020-04-10', '2020-04-10 10:00:00', -3.29, 0.09, -0.74, 0),
 (10, '2020-04-10', '2020-04-11 10:00:00', -4.58, 0.76, -0.16, 0),
 (11, '2020-04-10', '2020-04-12 10:00:00', -2.9, 1.56, 1.45, 2.71),
 (12, '2020-04-10', '2020-04-13 10:00:00', -3.06, -0.3, -0.56, 0),
 (13, '2020-04-10', '2020-04-14 10:00:00', -3.72, -0.44, -1.32, 0),
 (14, '2020-04-10', '2020-04-15 10:00:00', -3.69, 1.88, 1.47, 0),
 (15, '2020-04-10', '2020-04-16 10:00:00', -2.33, 4.8, 4.02, 0),
 (16, '2020-04-10', '2020-04-17 10:00:00', -0.23, 4.82, 3.76, 0),
 (17, '2020-04-11', '2020-04-11 10:00:00', -3.98, 1.31, 1.31, 0),
 (18, '2020-04-11', '2020-04-12 10:00:00', -3.29, 2.24, 1.43, 1.72),
 (19, '2020-04-11', '2020-04-13 10:00:00', -2.15, 1.61, 0.44, 0),
 (20, '2020-04-11', '2020-04-14 10:00:00', -2.62, 1.94, 1.71, 0),
 (21, '2020-04-11', '2020-04-15 10:00:00', -1.22, 1.9, 0.82, 0),
 (22, '2020-04-11', '2020-04-16 10:00:00', -1.68, 5.28, 3.59, 0),
 (23, '2020-04-11', '2020-04-17 10:00:00', 1.45, 5.35, 4.21, 2.06),
 (24, '2020-04-11', '2020-04-18 10:00:00', 0.65, 8.56, 6.97, 0.29),
 (25, '2020-04-12', '2020-04-12 10:00:00', 0, 3.73, 0, 6.64),
 (26, '2020-04-12', '2020-04-13 10:00:00', -1.77, 2.22, 1.09, 0),
 (27, '2020-04-12', '2020-04-14 10:00:00', -2.31, 2.71, 1.8, 0),
 (28, '2020-04-12', '2020-04-15 10:00:00', -1.93, 4, 3.17, 0),
 (29, '2020-04-12', '2020-04-16 10:00:00', -1.76, 3.55, 3.55, 0),
 (30, '2020-04-12', '2020-04-17 10:00:00', -2.24, 1.24, 1.24, 0),
 (31, '2020-04-12', '2020-04-18 10:00:00', -0.36, 3.61, 2.25, 0),
 (32, '2020-04-12', '2020-04-19 10:00:00', 0.63, 3.34, 2.97, 0),
 (33, '2020-04-13', '2020-04-13 10:00:00', -1.76, 1.98, 0.6, 0),
 (34, '2020-04-13', '2020-04-14 10:00:00', -2.53, 2.42, 1, 0),
 (35, '2020-04-13', '2020-04-15 10:00:00', -1.64, 3.53, 2.58, 0),
 (36, '2020-04-13', '2020-04-16 10:00:00', -0.32, 2.42, 2.28, 0.22),
 (37, '2020-04-13', '2020-04-17 10:00:00', 0.52, 3.68, 3.47, 2.98),
 (38, '2020-04-13', '2020-04-18 10:00:00', 0.33, 4.93, 3.4, 0),
 (39, '2020-04-13', '2020-04-19 10:00:00', -0.01, 3.44, 2.73, 0),
 (40, '2020-04-13', '2020-04-20 10:00:00', -1.25, 5.2, 2.31, 0),
 (41, '2020-04-14', '2020-04-14 10:00:00', -4, 3.13, 2.03, 0),
 (42, '2020-04-14', '2020-04-15 10:00:00', -2.28, 3.68, 3.02, 0),
 (43, '2020-04-14', '2020-04-16 10:00:00', -1.84, 3.62, 2.86, 0),
 (44, '2020-04-14', '2020-04-17 10:00:00', -1.19, 1.01, 0.46, 0),
 (45, '2020-04-14', '2020-04-18 10:00:00', -0.31, 2, 1.12, 0),
 (46, '2020-04-14', '2020-04-19 10:00:00', -2.36, 2.06, 0.89, 0),
 (47, '2020-04-14', '2020-04-20 10:00:00', -1.3, 4.64, 3.22, 0),
 (48, '2020-04-14', '2020-04-21 10:00:00', -1.73, 2.91, 2.74, 0),
 (49, '2020-04-15', '2020-04-15 10:00:00', -2, 2.99, 1.48, 0),
 (50, '2020-04-15', '2020-04-16 10:00:00', -2.48, 3.5, 2.67, 0),
 (51, '2020-04-15', '2020-04-17 10:00:00', -0.66, 1.57, 1.23, 0),
 (52, '2020-04-15', '2020-04-18 10:00:00', -0.63, 2.3, 0.6, 0),
 (53, '2020-04-15', '2020-04-19 10:00:00', -1.25, 1.94, 1.14, 0),
 (54, '2020-04-15', '2020-04-20 10:00:00', -0.9, 3.92, 2.14, 0),
 (55, '2020-04-15', '2020-04-21 10:00:00', -0.87, 5.38, 3.6, 0),
 (56, '2020-04-15', '2020-04-22 10:00:00', 0.81, 0.81, 0.81, 0),
 (57, '2020-04-16', '2020-04-16 10:00:00', -0.37, 1.23, 0, 0),
 (58, '2020-04-16', '2020-04-17 10:00:00', -1.52, 1.22, 0.62, 0),
 (59, '2020-04-16', '2020-04-18 10:00:00', -0.53, 2.39, 1.77, 0),
 (60, '2020-04-16', '2020-04-19 10:00:00', -1.87, 3.27, 2.25, 0),
 (61, '2020-04-16', '2020-04-20 10:00:00', 1.24, 6.03,